# Preprocessing Notebooks

Beberapa preprocessing untuk membuat skenario sebagai penelitian

1. Sama seperti preprosesing unutk membuat IndoBERTweet (lowercasing, change user, change emoji, change http url)
2. Nomor 1 dan melakukan stemming
3. Lowecasing, Menghilangkan atribut url dan mention dalam text, change emoji
4. Nomor 3 dan melakukan stemming

## Instantiate some processing class

In [2]:
from NDETCStemmer import NDETCStemmer, CustomModelDownloader
from kaelib.processor.NDETCStemmerWraper import NDETCStemmerWraper

downloader = CustomModelDownloader(
    model_1="https://is3.cloudhost.id/s3.kaenova.my.id/NDETCStemmer/Model/w2vec_wiki_id_case",
    model_2="https://is3.cloudhost.id/s3.kaenova.my.id/NDETCStemmer/Model/w2vec_wiki_id_case.trainables.syn1neg.npy",
    model_3="https://is3.cloudhost.id/s3.kaenova.my.id/NDETCStemmer/Model/w2vec_wiki_id_case.wv.vectors.npy"
)

original_stemmer=NDETCStemmer(custom_downloader=downloader)
stemmer = NDETCStemmerWraper(original_stemmer)

Model missing, downloading new model....



d:\@Programming\@On Going\Data\Headline_Detection\venv\lib\site-packages\NDETCStemmer\Model\w2vec_wiki_id_case: 100%|██████████| 25.5M/25.5M [00:07<00:00, 3.67MiB/s]


d:\@Programming\@On Going\Data\Headline_Detection\venv\lib\site-packages\NDETCStemmer\Model\w2vec_wiki_id_case.trainables.syn1neg.npy: 100%|██████████| 312M/312M [01:35<00:00, 3.42MiB/s] 


d:\@Programming\@On Going\Data\Headline_Detection\venv\lib\site-packages\NDETCStemmer\Model\w2vec_wiki_id_case.wv.vectors.npy: 100%|██████████| 312M/312M [01:34<00:00, 3.47MiB/s] 



Download Complete


In [3]:
from kaelib.processor.TextProcessingPipeline import TextProcessingPipeline
import kaelib.processor.preprocessing_func as pf

# Scenario one
scenario_1_processor = TextProcessingPipeline([
    pf.lowercasing,
    pf.change_user,
    pf.change_emoji,
    pf.change_web_url
])

# Scenario two
scenario_2_processor = TextProcessingPipeline([
    pf.lowercasing,
    pf.change_user,
    pf.change_emoji,
    pf.change_web_url,
    stemmer.stem
])

# Scenario three
scenario_3_processor = TextProcessingPipeline([
    pf.lowercasing,
    pf.remove_username,
    pf.remove_url,
    pf.change_emoji,
])

# Scenario four
scenario_4_processor = TextProcessingPipeline([
    pf.lowercasing,
    pf.remove_username,
    pf.remove_url,
    pf.change_emoji,
    stemmer.stem
])

test_text = """Miliarder Rusia Oleg Tinkov pada Senin (31/10/2022), mengaku telah  melepaskan kewarganegaraan Rusianya karena konflik di Ukraina. http://dlvr.it/Sc20gN @kaenova """

print("Example")
print("Scenario 1:", scenario_1_processor.process_text(test_text))
print("Scenario 2:", scenario_2_processor.process_text(test_text))
print("Scenario 3:", scenario_3_processor.process_text(test_text))
print("Scenario 4:", scenario_4_processor.process_text(test_text))

Example
Scenario 1: miliarder rusia oleg tinkov pada senin (31/10/2022), mengaku telah  melepaskan kewarganegaraan rusianya karena konflik di ukraina. HTTPURL @USER 
Scenario 2: miliarder rusia oleg tinkov pada senin 31 10 2022 aku telah lepas warga negara rusianya karena konflik di ukraina HTTPURL @USER 
Scenario 3: miliarder rusia oleg tinkov pada senin (31/10/2022), mengaku telah  melepaskan kewarganegaraan rusianya karena konflik di ukraina.   
Scenario 4: miliarder rusia oleg tinkov pada senin 31 10 2022 aku telah lepas warga negara rusianya karena konflik di ukraina 


In [12]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

tqdm.pandas()

df_all = pd.read_csv("../../data/3. Labeling Ready/concat_all.csv")
df_all['tweet'] = df_all['tweet'].progress_map(lambda x: scenario_1_processor.process_text(x))
X = df_all['tweet']
y = df_all['labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

pd.DataFrame({
    "tweet": X_train,
    "labels": y_train
}).to_csv("../../data/4. Processed/train.csv", index=False)


pd.DataFrame({
    "tweet": X_test,
    "labels": y_test
}).to_csv("../../data/4. Processed/test.csv", index=False)

100%|██████████| 2328/2328 [00:00<00:00, 15520.36it/s]


In [49]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
datasets = HeadlineDataset(
    "../../data/4. Processed/train.csv",
    tokenizer,
    max_length=256,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)
loader = DataLoader(dataset=datasets, batch_size=32)